In [1]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import pandas as pd
import numpy as np
import nbimporter
#import preprocessing # import Jupyter notebook
#from preprocessing import clean_pdf
import clean_data
import gensim.corpora as corpora
import collections


In [2]:
import tomotopy as tp
print(tp.isa) # prints 'avx2', 'avx', 'sse2' or 'none'

avx2


In [3]:
# Load full dataset
# full_data = './Dataset/merged/textdata_all.pkl'
final_corpus = './Dataset/merged/final_corpus.pkl'
with open(final_corpus, 'rb') as f:
    fc_df = pickle.load(f)
# text_df.reset_index(drop=True, inplace=True)

In [4]:
docs_cleaned = clean_data.clean_pdf(fc_df)

0.16258597373962402
0.1627640724182129
0.16277790069580078
0.16279292106628418
0.1628119945526123
0.16282987594604492
Tokenizing
44.19356989860535
44.19361114501953
44.19361901283264
Lemmatizing
44.19363307952881
Bag of Words Representation
length of dct before filter_extreme:  156733
length of dct after filter_extreme:  10735
157.63465309143066


In [5]:
docs_cleaned_dir = './Dataset/cleaned/final/docs/'
docs_cleaned_file = docs_cleaned_dir + 'docs_cleaned.pkl'
os.makedirs(docs_cleaned_dir,exist_ok=True)
with open(docs_cleaned_file, 'wb') as f:
    pickle.dump(docs_cleaned,f)

## Cleaning sections

In [6]:
sections_df = clean_data.process_sections(fc_df)
sections_df.head(10)

,whole_text,section_titles,paper_id
0,"Throughout the technical note, we use capital ...",II. PRELIMINARIES,18980380
1,1) The confidence set O ns s is bounded and ha...,Assumption 3 (Regularity Conditions forC s ):,18980380
2,This section focuses on DAMDP with a finite nu...,III. FINITE HORIZON DISTRIBUTIONALLY ROBUST MDPS,18980380
3,"In this section, we study two synthetic numeri...",IV. SIMULATION,18980380
4,We consider a machine replacement problem simi...,A. Reward Uncertainty in the Machine Replaceme...,18980380
5,We first consider a machine replacement proble...,1) Machine Replacement as a MDP With Gaussian ...,18980380
6,The second experiment has a similar setup as t...,2) Machine Replacement as a MDP With Mixed Gau...,18980380
7,"We now consider a path planning problem, simil...",B. Transition Uncertainty in the Path Planning...,18980380
0,Morley (1991) has explained that in developing...,A. Listening Materials and Activities,56031008
1,"Prior knowledge in listener""s mind entails the...",B. Schema Theory and Background Knowledge,56031008


In [23]:
sections_cleaned = clean_data.clean_section(sections_df)

changing index to paper_id
0.8703620433807373
0.8704631328582764
0.870474100112915
0.8704822063446045
0.8704900741577148
0.8704988956451416
Tokenizing
41.65893006324768
41.658942222595215
41.658950090408325
Lemmatizing
41.65896487236023
Bag of Words Representation
length of dct before filter_extreme:  144802
length of dct after filter_extreme:  13165
152.77748107910156


In [24]:
len(sections_cleaned['dct'])

13165

In [25]:
sections_cleaned_dir = './Dataset/cleaned/final/sections/'
sections_cleaned_file = sections_cleaned_dir + 'sections_cleaned.pkl'
os.makedirs(sections_cleaned_dir,exist_ok=True)
with open(sections_cleaned_file,'wb') as f:
    pickle.dump(sections_cleaned,f)

In [ ]:
corpus = sections_cleaned['corpus']
dct = sections_cleaned['dct']
paper_ids = sections_cleaned['ids']
docs = sections_cleaned['docs']

In [34]:
corpus_doc = docs_cleaned['corpus']
dct_doc = docs_cleaned['dct']
paper_ids_doc = docs_cleaned['ids']
docs_doc = docs_cleaned['docs']

In [14]:
# Calculate weights
weights = np.zeros((len(corpus), len(dct.cfs)))
for d_idx,d in enumerate(corpus):

    #print(d_idx)
    for w_idx, c in enumerate(d):
        
        # section length
        section_length = sum([x[1] for x in d])
        
        # Calculate section-level weight
        weight_section = (c[1] + 1) / (dct.cfs[w_idx] + section_length)
        
        # Calculate document-level weight
        paper_id = paper_ids[d_idx]
        ind = paper_ids_doc.index(paper_id)
        
        # document length
        document_length = sum([x[1] for x in corpus_doc[ind]])
        
        w_idx_doc = dct_doc.token2id[dct[w_idx]]
        
        count_doc = 0
        if w_idx_doc in dict(corpus_doc[ind]).keys():
            count_doc = dict(corpus_doc[ind])[w_idx_doc]
        
        
        weight_document = (count_doc + 1) / (dct_doc.cfs[w_idx_doc] + document_length)
        
        # Combined weight
        weight_combined = weight_section*weight_document
    
        weights[d_idx][w_idx] = weight_combined
weights = (weights-np.min(weights)) / (np.max(weights)-np.min(weights))

NameError: name 'corpus' is not defined

In [273]:
# Update gensim corpus with weighted values
for d_idx,d in enumerate(corpus):

    for w_idx, c in enumerate(d):
    
        corpus[d_idx][w_idx] = (c[0], float(c[1])*weights[d_idx][w_idx])

## LDA Model on Document level data

In [15]:
mdl = tp.LDAModel(k=20)
ndocs = len(docs_cleaned['docs'])
for i in range(ndocs):
    mdl.add_doc(docs_cleaned['docs'][i])

In [23]:
a = list(mdl.docs)

In [27]:
# help(mdl)

In [16]:
for i in range(0, 100, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))

mdl.summary()

Iteration: 0	Log-likelihood: -9.52556707920659
Iteration: 10	Log-likelihood: -8.886778533761598
Iteration: 20	Log-likelihood: -8.749836794938812
Iteration: 30	Log-likelihood: -8.685682991883231
Iteration: 40	Log-likelihood: -8.647496887347124
Iteration: 50	Log-likelihood: -8.621290559515279
Iteration: 60	Log-likelihood: -8.602533973124823
Iteration: 70	Log-likelihood: -8.590162400043251
Iteration: 80	Log-likelihood: -8.578226996558925
Iteration: 90	Log-likelihood: -8.56970758550309
Top 10 words of topic #0
[('channel', 0.029091021046042442), ('system', 0.01907355710864067), ('signal', 0.01591496169567108), ('performance', 0.013407456688582897), ('antenna', 0.012973396107554436), ('proposed', 0.009926006197929382), ('number', 0.008652524091303349), ('power', 0.007875879295170307), ('mimo', 0.007727006915956736), ('symbol', 0.007357517722994089)]
Top 10 words of topic #1
[('image', 0.031919896602630615), ('feature', 0.01562705636024475), ('method', 0.01201371755450964), ('layer', 0.00734

In [55]:
mdl.perplexity, mdl.

AttributeError: 'tomotopy.LDAModel' object has no attribute 'coherence'

## LDA Model on Section level data

In [288]:
mdl_section = tp.LDAModel(k=20)
ndocs = len(sections_cleaned['docs'])
for i in range(ndocs):
    mdl_section.add_doc(sections_cleaned['docs'][i])

In [289]:
for i in range(0, 100, 10):
    mdl_section.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl_section.ll_per_word))

for k in range(mdl_section.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl_section.get_topic_words(k, top_n=10))

mdl_section.summary()

Iteration: 0	Log-likelihood: -9.079915884555385
Iteration: 10	Log-likelihood: -8.701078695364467
Iteration: 20	Log-likelihood: -8.564733159888059
Iteration: 30	Log-likelihood: -8.485860552421332
Iteration: 40	Log-likelihood: -8.437313880667375
Iteration: 50	Log-likelihood: -8.396943245529172
Iteration: 60	Log-likelihood: -8.368769717149952
Iteration: 70	Log-likelihood: -8.346425243730684
Iteration: 80	Log-likelihood: -8.323723462836833
Iteration: 90	Log-likelihood: -8.30783522650672
Top 10 words of topic #0
[('control', 0.04231368750333786), ('system', 0.03068280778825283), ('method', 0.018112976104021072), ('controller', 0.013630240224301815), ('tracking', 0.01278215553611517), ('state', 0.012388401664793491), ('model', 0.01217638049274683), ('target', 0.011812916025519371), ('wind', 0.011722049675881863), ('speed', 0.01126771792769432)]
Top 10 words of topic #1
[('student', 0.023132802918553352), ('test', 0.012613781727850437), ('learning', 0.011423846706748009), ('teacher', 0.009377

In [295]:
## Modify corpus: adding in weights

array([1216, 1094, 1151, ...,    1,    1,    1], dtype=uint64)

In [29]:
# mdl_section.perplexity

## LDA model with gensim

In [51]:
# Step 4: Train the LDA model
from gensim.models import LdaModel, LdaMulticore
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models.callbacks import PerplexityMetric, ConvergenceMetric, CoherenceMetric
import logging

field = 'cs-med-kw-only'
num_topics = 20
model_dir= f'./models/{field}/k_{num_topics}/'
os.makedirs(model_dir,exist_ok=True)

model_file = model_dir + 'model'
# The filename is the file that will be created with the log.
# If the file already exists, the log will continue rather than being overwritten.
log_file = model_dir + 'model_callbacks.log'
# log_file = 'model_callbacks_test.log'
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename=log_file,
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.DEBUG)

perplexity_logger = PerplexityMetric(corpus=corpus_doc, logger='shell')
convergence_logger = ConvergenceMetric(logger='shell')
coherence_cv_logger = CoherenceMetric(corpus=corpus_doc, logger='shell', coherence = 'c_v', texts = docs_doc) # very compute intensive
coherence_umass_logger = CoherenceMetric(corpus=corpus_doc, logger='shell', coherence = 'u_mass')

lda_model = LdaModel(corpus=corpus_doc,
                         id2word=dct_doc,
                         random_state=2020,
                         num_topics=num_topics,
                         passes=1,
                         chunksize=1000,
#                          batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta='auto',
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True,
                         callbacks=[perplexity_logger,
                                    convergence_logger])
#                                     coherence_cv_logger,
                                    #coherence_umass_logger])

lda_model.save(model_file)

In [52]:
from pprint import pprint
pprint(lda_model.print_topics())

[(0,
  '0.011*"image" + 0.010*"feature" + 0.007*"method" + 0.007*"model" + '
  '0.006*"used" + 0.005*"set" + 0.005*"using" + 0.005*"data" + 0.005*"result" '
  '+ 0.005*"training"'),
 (1,
  '0.007*"channel" + 0.007*"algorithm" + 0.007*"problem" + 0.005*"power" + '
  '0.005*"set" + 0.005*"number" + 0.005*"system" + 0.005*"function" + '
  '0.004*"case" + 0.004*"one"'),
 (2,
  '0.002*"data" + 0.002*"time" + 0.001*"system" + 0.001*"power" + '
  '0.001*"method" + 0.001*"value" + 0.001*"result" + 0.001*"network" + '
  '0.001*"algorithm" + 0.001*"based"'),
 (3,
  '0.002*"user" + 0.002*"power" + 0.002*"channel" + 0.002*"algorithm" + '
  '0.002*"system" + 0.002*"data" + 0.001*"number" + 0.001*"time" + 0.001*"two" '
  '+ 0.001*"proposed"'),
 (4,
  '0.001*"method" + 0.001*"system" + 0.001*"data" + 0.001*"time" + '
  '0.001*"value" + 0.001*"number" + 0.001*"result" + 0.001*"two" + 0.001*"set" '
  '+ 0.001*"different"'),
 (5,
  '0.003*"method" + 0.003*"network" + 0.003*"model" + 0.003*"feature" + '


NameError: name 'corpus' is not defined

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus_doc, texts = docs_doc, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [276]:
def get_top_documents(corpus,ids,topic):
    num_topics = 5
    topic_proportions = np.zeros(shape=(len(corpus),num_topics+1))
    topic_proportions[:,0] = ids
    response = lda_model.get_document_topics(corpus)
    for i,doc in enumerate(response):
        for t,prop in doc:
            topic_proportions[i][t+1] = prop 
    ind = topic_proportions[:,topic+1].argsort()[::-1]
    return topic_proportions[ind]